#ALL FUNCTIONS

##ADD Libraries

In [ ]:
import os
import datetime
import re
import pandas as pd
import json
import xml.etree.ElementTree as et
import csv
from datetime import datetime,timedelta
import pickle
from io import BytesIO
import sys
import requests
import time
import math

## Class to convert Scripts from dev to prod and vice versa

In [ ]:
class code_conversions:
  load_sequence=['planisware_finance_data.sql',
'gc_config.sql',
'child_gc_config.sql',
'child_gc_config_split.sql',
'grand_gc_config.sql',
'child_gc_req_set.sql',
'defaultworkflow.sql',
'vox_defaultworkflow.sql',
'standard_attribute.sql',
'unsatisfied_stakeholder_requirements.sql',
'simulation.sql',
'gcid_appitem_list.sql',
'voe_non_dng.sql',
'voe_requirements.sql',
'vox_requirements.sql',
'rqm.sql',
'rqm_log.sql',
'percent_test_plan_verified_phase2.sql',
'test_plan_phase2_split.sql',
'test_plan_verified_phase2_latest.sql',
'percentage_phase2.sql',
'percentage_phase2_split.sql',
'percentage_phase2_latest.sql',
'trend_cv.sql',
'trend_cvdv.sql',
'trend_dv.sql',
'trend_pv.sql',
'interface_variant_report.sql',
'interface_variant_requirements_report.sql',
'interface_variant_requirements_report_latest.sql',
'icd_requirement_report.sql',
'icd_requirement_report_latest.sql',
'simulation_validation.sql',
'simulation_validation_latest.sql',
'test_script_maturity.sql',
'test_script_maturity_latest.sql',
'test_case_attribute.sql',
'test_case_without_req.sql',
'test_case_with_req.sql',
'test_result.sql',
'test_plan_attribute.sql',
'test_plan_yield.sql',
'sbd platform hardcoded.sql',
'orphan_tcer_table.sql',
'platform_orphan_table.sql',
'orphan_monthly.sql',
'project_portfolio.sql',
'project_portfolio_summary_test_plan.sql',
'ppm_gc_view.txt',
'active_and_library_gc_configuration.txt',
'voe_vox_defaultworkflow_view.txt',
'vandv_rqm.txt',
'simulation_tcer_view.txt',
'labels_view.txt',
'KPI_Query.txt',
'KPI_Query_vox.txt',
'null_voer_identifiers.txt',
'interface_variant_report_summary.txt',
'icd_requirement_report_summary.sql',
'interface_variant_requirements_report_config_table.txt',
'interface_variant_requirement_report_compliance.sql',
'interface_variant_requirement_report_config.txt',
'interface_variant_requirement_report_compliance_table.txt',
'availabiliy_by_attribute.sql',
'simulation_validation_latest_availability.txt',
'availability_summary.sql',
'vox_total_mismatch.txt',
'test Plan CV+DV.txt',
'test_plan_verfied_cvdv_summary.txt',
'test_plan_verified_phase2_test_verdict_phase_validation.txt',
'test_plan_verified_cvdv_tableview.txt',
'orphan_tcer.sql',
'coverage.sql',
'fmea.txt',
'physical_sbd.sql',
'test_script_maturity_avoidance.sql',
'avoidance_missing.sql',
'Portfolio.sql',
'platform_orphan_tcer.sql',
'Vpd_project_metrics.sql',
'coverage_orphan.sql',
'product_category_platfrom_hardocoded.sql',
'vpd_project_metrics_notorphan.sql',
'vpd_project_metrics_notorphan_table.sql',
'orphan_test_case.sql',
'orphan_test_case_table.sql',
'orphan_tcer_monthly_summary.sql',
'vpd_monthly.sql']

  def __init__(self,convert_to_env='dev',replace_converted_script=True,populate_dateparm=False,expand_bulk_load_func=False,
               expand_table_setup_func=False,
               unload_src='prd',unload_s3='qa',unload_add_created_on=True,unload_d_ate='',unload_dynamic_timestamp=False):
    self.directory_sequence=[]
    self.file_sequence=[]
    self.convert_to_env=convert_to_env
    self.replace_converted_script=replace_converted_script
    self.populate_dateparm=populate_dateparm
    self.unload_src=unload_src
    self.unload_s3=unload_s3
    self.unload_add_created_on=unload_add_created_on
    self.unload_d_ate=unload_d_ate
    self.unload_dynamic_timestamp=unload_dynamic_timestamp
    self.dateparm=(datetime.now()- timedelta(days=1)).strftime('%Y-%m-%d')
    self.expand_bulk_load_func=expand_bulk_load_func
    self.expand_table_setup_func=expand_table_setup_func



  def get_sequence(self):
    self.directory_sequence=os.listdir('.')
    for expected_sequence in self.load_sequence:
      expected_sequence=expected_sequence.split('.')[0]
      for directory_sequence in self.directory_sequence:
        directory_sequence_split=directory_sequence.split('.')[0]
        if directory_sequence_split==expected_sequence:
          self.file_sequence.append(directory_sequence)
    print(self.file_sequence)

  def remove_dup_files(self):
    self.get_sequence()
    for files in self.directory_sequence:
      if ('.sql' in files or '.txt' in files or '.csv' in files or '.json' in files) and (not files.startswith(".")) and '_output' not in files:
        os.remove(files)
    for files in self.directory_sequence:
      if ('.sql' in files or '.txt' in files) and (not files.startswith(".")) and '_output' in files:
        os.replace(files,files.replace('_output',''))

  def combine_files(self,name):
    self.get_sequence()
    df=pd.DataFrame()
    for f in self.directory_sequence:
      if f.endswith(".csv"):
        df_sample=pd.read_csv(f)
        df=df.append(df_sample)
    name = name + '.csv'
    if self.replace_converted_script==True:
      self.remove_dup_files()
    df.to_csv(name,index=False)


  def create_exec_file(self):
    view_name=''
    self.get_sequence()
    for file in self.directory_sequence:
      if '.sql' in file or '.txt' in file:
        with open(file, 'r') as f:
          lines = f.readlines()
        name=((file).replace('.txt','')).replace('.sql','')
        out = open(name+'_output.sql','w')

        for line in lines:
          if self.convert_to_env == 'qa' or self.convert_to_env == 'prd':
            line = line.replace('dev.data_', 'data.gpo_')
            line = line.replace('dev.stg1_', 'etl.stg1_gpo_')
            line = line.replace('dev.stg2_', 'etl.stg2_gpo_')
            line = line.replace('dev.stg3_', 'etl.stg3_gpo_')
            line = line.replace('dev.stg4_', 'etl.stg4_gpo_')
            line = line.replace('dev.mbse_', 'gpo.mbse_')
            line = line.replace('inserted_at','etl_timestamp')
            line = line.replace('dev.table_setup','etl.table_setup')
            line = line.replace('dev.bulk_load','etl.bulk_load')
            if self.expand_bulk_load_func==True:
              line=self.expand_bulk_load(line)
            if self.expand_table_setup_func==True:
              line=self.expand_table_setup(line)
            line = line.replace('arn:aws:iam::233694013590:role/redshift-worker','arn:aws:iam::635540041123:role/redshift-worker')
            if self.convert_to_env == 'prd':
              line = line.replace('datalake-dev-s3-data/gpo-', 'datalake-env-s3-data/gpo-')
              line = line.replace('datalake-qa-s3-data/gpo-', 'datalake-env-s3-data/gpo-')
            elif self.convert_to_env == 'qa':
              line = line.replace('datalake-dev-s3-data/gpo-', 'datalake-qa-s3-data/gpo-')
              line = line.replace('datalake-env-s3-data/gpo-', 'datalake-qa-s3-data/gpo-')

          if self.convert_to_env == 'dev':
            line = line.replace('data.gpo_','dev.data_')
            line = line.replace('etl.stg1_gpo_','dev.stg1_')
            line = line.replace('etl.stg2_gpo_','dev.stg2_')
            line = line.replace('etl.stg3_gpo_','dev.stg3_')
            line = line.replace('etl.stg4_gpo_','dev.stg4_')


            '''
            if line.startswith('DROP VIEW IF EXISTS') or line.startswith('drop view if exists'):
              view_name = line
              view_name = line.split('exists')[1]
              view_name = view_name.split('.')[1]
              view_name = view_name.split(';')[0]
            '''


            line = line.replace('gpo.mbse_', 'dev.mbse_')
            line = line.replace('inserted_at','etl_timestamp')
            line = line.replace('etl.table_setup','dev.table_setup')
            line = line.replace('etl.bulk_load','dev.bulk_load')
            if self.expand_bulk_load_func==True:
              line=self.expand_bulk_load(line)
            if self.expand_table_setup_func==True:
              line=self.expand_table_setup(line)
            line = line.replace('datalake-qa-s3-data/gpo-','datalake-dev-s3-data/gpo-')
            line = line.replace('datalake-env-s3-data/gpo-','datalake-dev-s3-data/gpo-')
            line = line.replace('arn:aws:iam::635540041123:role/redshift-worker','arn:aws:iam::233694013590:role/redshift-worker')
            '''
            if line.lower().startswith('grant select on'):

              line = line.replace('GRANT SELECT ON ALL TABLES IN SCHEMA gpo TO GROUP gpo;','')
              line = f'grant all on dev.{view_name} to handoa1;'
            '''


          if self.populate_dateparm==True:
            match = re.search(r'{dateparm}', line)
            if match:
              line = re.sub(r'{dateparm}', self.dateparm, line)
          out.write(line)
        out.close()
    if self.replace_converted_script==True:
      self.remove_dup_files()


  def get_load(self,file):
    with open(file, 'r') as f:
        lines = f.readlines()
    load_copy=0
    load_preprocess=0
    load_insert=0
    res=''
    copy=''
    preprocess=''
    insert=''
    for line in lines:
      line=line.replace('  ',' ')
      if (line.lower().startswith("truncate table etl.stg1_gpo_mbse_") or line.lower().startswith("truncate table dev.stg1_mbse_") or
          line.lower().startswith("copy ") or line.startswith("call dev.bulk_load(") or line.startswith("call etl.bulk_load(")) and load_copy==0:
        load_copy=1
      if load_copy==1:
        if ";" in line:
          copy+=line+"\n"
          load_copy=0
        else:
          if self.populate_dateparm==True:
            match = re.search(r'{dateparm}', line)
            if match:
              line = re.sub(r'{dateparm}', self.dateparm, line)
          copy+=line

    for line in lines:
      line=line.replace('  ',' ')
      if (line.lower().startswith("truncate table etl.stg4_gpo_mbse_") or line.lower().startswith("truncate table dev.stg4_mbse_") or
          line.lower().startswith("call dev.table_setup('dev.stg4_mbse_") or line.lower().startswith("call etl.table_setup('etl.stg4_gpo_mbse_") or
          line.lower().startswith("insert into dev.stg4_mbse_") or line.lower().startswith("insert into etl.stg4_gpo_mbse_")) and load_preprocess==0:
        load_preprocess=1
      if load_preprocess==1:
        if ";" in line:
          preprocess+=line+"\n"
          load_preprocess=0
        else:
          preprocess+=line

    for line in lines:
      line=line.replace('  ',' ')
      if (line.lower().startswith("truncate table data.gpo_mbse_") or line.lower().startswith("truncate table dev.data_mbse_") or
          line.lower().startswith("call dev.table_setup('dev.data_mbse_") or line.lower().startswith("call etl.table_setup('data.gpo_mbse_") or
          line.lower().startswith("delete from") or
          line.lower().startswith("insert into dev.data_mbse_") or line.lower().startswith("insert into data.gpo_mbse_") or
          line.lower().startswith("update dev.data_mbse_") or line.lower().startswith("update data.gpo_mbse_")) and load_insert==0:
        load_insert=1
      if load_insert==1:
        if ";" in line:
          insert+=line+"\n"
          load_insert=0
        else:
          insert+=line
    return copy,preprocess,insert


  def get_ddl(self,file):
    with open(file, 'r') as f:
      lines = f.readlines()
    ddl_drop_view=0
    ddl_drop_table=0
    ddl_table=0
    ddl_view=0
    res_drop_table=''
    res_drop_view=''
    res_table=''
    res_view=''
    #View Drop Statements
    for line in lines:
      if line.lower().startswith("drop view") and ddl_drop_view==0:
        ddl_drop_view=1
      if ddl_drop_view==1:
        if ";" in line:
          res_drop_view+=line+"\n"
          ddl_drop_view=0
        else:
          res_drop_view+=line

  #Table Drop Statements
    for line in lines:
      if line.lower().startswith("drop table") and ddl_drop_table==0:
        ddl_drop_table=1
      if ddl_drop_table==1:
        if ";" in line:
          res_drop_table+=line+"\n"
          ddl_drop_table=0
        else:
          res_drop_table+=line

    #Create Table statements
    for line in lines:
      if (line.lower().startswith("create table ") or line.lower().startswith("create or replace table ") or
          line.lower().startswith("alter table ")) and ddl_table==0:
        ddl_table=1
      if ddl_table==1:
        if ";" in line:
          res_table+=line+"\n"
          ddl_table=0
        else:
          res_table+=line

  #Create View statements
    for line in lines:
      if (line.lower().startswith("create view ") or line.lower().startswith("create or replace view ")) and ddl_view==0:
        ddl_view=1
      if ddl_view==1:
        if ";" in line:
          res_view+=line+"\n"
          ddl_view=0
        else:
          res_view+=line
    return res_drop_view,res_drop_table,res_table,res_view


  def get_staging_columns(self,file):
    with open(file,'r') as f:
      lines = f.readlines()
    take=0
    take1=0
    res=''
    table=''
    path=''
    for line in lines:

      #Extract S3 Path
      if "copy" in line or take1==1:
        take1+=1
      if take1==2 and line.startswith("from"):
        path=line.replace('from','')
        take1=0
      #Extract Table Columns
      if "CREATE TABLE etl.stg1" in line:
        table=line.replace('CREATE TABLE ','').replace("(",'')
      if "CREATE TABLE etl.stg1" in line or take==1:
        take+=1
      if line.startswith(")"):
        take=0
      if take==2:
        res+=(line.split(' ')[0]).split('\t')[0]+","+"\n"
    res=res[:-2]
    return res,table,path


  def unload(self,file):
    from datetime import date
    date = date.today()
    l_ines,tbl,path=self.get_staging_columns(file)

    if self.unload_src=='dev':
      tbl=tbl.replace("etl.stg1_gpo_mbse_","dev.data_mbse_")
    elif self.unload_src=='qa':
      tbl=tbl.replace("etl.stg1_gpo_mbse_","gpo.mbse_")
    elif self.unload_src=='prd':
      tbl=tbl.replace("etl.stg1_gpo_mbse_","gpo.mbse_")

    if self.unload_s3=='dev':
      id=233694013590
      if self.unload_dynamic_timestamp  == False:
        path=path[:-2].replace('env','dev')+'back_'+str(date)+"_"
      else:
        path = path[:-2].replace('env','dev')+'back_'+"{dateparm}_"
    elif self.unload_s3=='qa':
      id=635540041123
      if self.unload_dynamic_timestamp  == False:
        path=path[:-2].replace('env','env')+'back_'+str(date)+"_"
      else:
        path = path[:-2].replace('env','env')+'back_'+"{dateparm}_"
    elif self.unload_s3=='prd':
      id=635540041123
      if self.unload_dynamic_timestamp  == False:
        path=path[:-2].replace('env','env')+'back_'+str(date)+"_"
      else:
        path = path[:-2].replace('env','env')+'back_'+"{dateparm}_"


    if self.unload_add_created_on==True:
      unld='unload ('+"'"+"select "+l_ines+" from "+tbl+" where created_on>''"+self.unload_d_ate+"'''"
    else:
      unld='unload ('+"'"+"select "+l_ines+" from "+tbl+"'"
    to_path=") to\n"+path+"'\niam_role 'arn:aws:iam::"+str(id)+":role/redshift-worker' csv header parallel off;"

    if l_ines=='' or tbl=='' or path=='':
      unload_stmnt="\n"
      unload_stmnt_scott="\n"
    else:
      unload_stmnt=unld+to_path
    return unload_stmnt

  def get_load_ddl_unload(self):
    from datetime import date
    date = date.today()
    arr_copy=''
    arr_preprocess=''
    arr_insert=''
    res_unload=''
    drop_view_ddl=''
    drop_table_ddl=''
    table_ddl=''
    view_ddl=''
    self.get_sequence()
    for file in self.file_sequence:
      if ('.sql' in file or '.txt' in file) and (not file.startswith(".")):
        c,p,i = self.get_load(file)
        arr_copy+=c+"\n"
        arr_preprocess+=p+"\n"
        arr_insert+=i+"\n"

        dv,dt,t,v=self.get_ddl(file)
        drop_view_ddl+=dv+"\n"
        drop_table_ddl+=dt+"\n"
        table_ddl+=t+"\n"
        view_ddl+=v+"\n"
        res_unload+="\n"+self.unload(file)+"\n"

    res_load=arr_copy+"\n"+arr_preprocess+"\n"+arr_insert
    res_ddl=drop_view_ddl+"\n"+drop_table_ddl+"\n"+table_ddl+"\n"+view_ddl+"\n"+"GRANT SELECT ON ALL TABLES IN SCHEMA gpo TO GROUP gpo;"
    return res_load,res_ddl,res_unload

  def convert_date(self,date_str):
    dt_object=datetime.fromisoformat(date_str[:-1])
    date = dt_object.date()
    time = dt_object.time()
    return date,time

  def write_file(self,data,name):
    out = open(name+'.sql','w')
    for line in data:
        out.write(line)
    out.close()
    return (name+'.sql')


  def extract_error_report(self):
    self.get_sequence()
    xml_path=[]
    csv_path=[]
    name=[]
    url=[]
    error_message=[]
    start_date=[]
    start_time=[]
    status=[]
    stop_date=[]
    stop_time=[]
    state_machine_arn=[]
    # Loop through all the JSON files in the directory
    for filename in self.directory_sequence:
      if filename.endswith(".json"):
        with open(filename, "r") as jsonfile:
          data = json.load(jsonfile)

        for i in range(len(data)):
          input_param=json.loads(data[i]['Input'])
          try:
            xml_path.append(input_param["xml_path"])
            csv_path.append(input_param["csv_path"])
            url.append(input_param["url"])
          except:
            xml_path.append('')
            csv_path.append('')
            url.append('')

          #Get Error message
          try:
            info=data[i]['Cause']
            s_tart = info.index('"errorMessage":"') + len('"errorMessage":"')
            e_nd = info.index('"}', s_tart)

            error_message.append(info[s_tart:e_nd])
          except:
            error_message.append('')

          strt_date,strt_time=self.convert_date(data[i]['StartDate'])
          start_date.append(strt_date)
          start_time.append(strt_time)
          status.append(data[i]['Status'])

          stp_date,stp_time=self.convert_date(data[i]['StopDate'])
          stop_date.append(stp_date)
          stop_time.append(stp_time)
          name.append(data[i]["Name"])
          state_machine_arn.append(data[i]['StateMachineArn'])

    df=pd.DataFrame({'xml_path':xml_path,
                    'csv_path':csv_path,
                    'url':url,
                    'error_message':error_message,
                    'start_date':start_date,
                    'start_time':start_time,
                    'status':status,
                    'stop_date':stop_date,
                    'stop_time':stop_time,
                    'name':name,
                    'state_machine_arn':state_machine_arn
                    })
    df['csv_path']=df['csv_path'].str.replace('^s3://datalake-prd-s3-data/gpo-mbse-datalake/', '', regex=True)
    #print(df.head())
    split_df = df['csv_path'].str.split('/', expand=True)
    df['Report_Name']=split_df[0]
    df['File_Name']=split_df[1]
    df['start_date'] = df['start_date'].astype(str)
    df['stop_date'] = df['stop_date'].astype(str)
    df['start_time'] = df['start_time'].astype(str)
    df['stop_time'] = df['stop_time'].astype(str)
    df['start_datetime'] = pd.to_datetime(df['start_date'] +' '+ df['start_time'])
    df['stop_datetime'] = pd.to_datetime(df['stop_date'] +' '+ df['stop_time'])
    df['time_delta'] = df['stop_datetime'] - df['start_datetime']
    df['time_delta'] = df['time_delta'].dt.total_seconds().apply(lambda x: '{:02}:{:02}:{:02}'.format(int(x // 3600), int((x % 3600) // 60), int(x % 60)))
    df=df[['Report_Name','File_Name','status','start_date','start_time','stop_date','stop_time','time_delta']]
    df.to_csv('error_report.csv',index=False)

  def expand_bulk_load(self,txt):
    if 'ppm/data/' in txt.lower():
      txt=txt.lower().replace("call dev.bulk_load(","").replace("call etl.bulk_load(","").replace(")","").replace(";","").split(',')
      line="TRUNCATE TABLE "+str(txt[1]).strip().replace("'","")+";\n"
      line=line+"copy "+str(txt[1]).strip().replace("'","")
      line=line+"\n from "+txt[0]
      line=line+"\n iam_role 'arn:aws:iam::233694013590:role/redshift-worker' \n DELIMITER AS '|' \n"
      line=line+"DATEFORMAT 'auto' TIMEFORMAT 'auto'\n"
      line=line+"FORMAT csv EMPTYASNULL BLANKSASNULL ACCEPTINVCHARS TRUNCATECOLUMNS IGNOREBLANKLINES IGNOREHEADER 1 MAXERROR as 1;"
      print('in ppm')
      return line
    elif txt.lower().startswith("call dev.bulk_load(") or txt.lower().startswith("call etl.bulk_load("):
      txt=txt.lower().replace("call dev.bulk_load(","").replace("call etl.bulk_load(","").replace(")","").replace(";","").split(',')
      line="TRUNCATE TABLE "+str(txt[1]).strip().replace("'","")+";\n"
      line=line+"copy "+str(txt[1]).strip().replace("'","")
      line=line+"\n from "+txt[0]
      line=line+"\n iam_role 'arn:aws:iam::635540041123:role/redshift-worker' \n DELIMITER AS ',' \n"
      line=line+"DATEFORMAT 'auto' TIMEFORMAT 'auto'\n"
      line=line+"FORMAT csv EMPTYASNULL BLANKSASNULL ACCEPTINVCHARS TRUNCATECOLUMNS IGNOREBLANKLINES IGNOREHEADER 1 MAXERROR as 1;"
      return line
    else:
      return txt




  def expand_table_setup(self,txt):
    if txt.lower().startswith("call dev.table_setup(") or txt.lower().startswith("call etl.table_setup("):
      txt=txt.lower().replace("call dev.table_setup(","").replace("call etl.table_setup(","").replace(")","").replace(";","")
      line="TRUNCATE TABLE "+str(txt).strip().replace("'","")+";\n"
      return line
    else:
      return txt

  def find_max_created_on(self):
    self.get_sequence()
    ch = 'select max(created_on),count(*) as cnt, '
    col_name=' as tbl'
    files = os.listdir('.')
    tbl=[]
    for file in self.directory_sequence:
        if ('.sql' in file or '.txt' in file) and (not file.startswith(".")) and '_output' not in file:
            with open(file, 'r') as f:
              lines = f.readlines()
              for line in lines:
                line=line.lower()
                '''
                LOAD SCRIPT EXTRACTION
                with open(file, 'w') as f:
                  for line in lines:
                    if line.lower().startswith('call etl.bulk_load'):
                      line =line.split(',')[1]
                      line=line.replace('')
                      ch = 'select max(created_on) from '
                      line= ch+line+';\n'
                    f.write(line)

                    if line.lower().startswith("insert into "):
                      line = line[12:]
                      line = line.split(' ')[0]
                      ch = 'select max(created_on) from '
                      line = ch+line+';\n'
                    f.write(line)
                '''
                if line.startswith("drop view if exists ") or line.startswith("drop table if exists ") or line.startswith("drop view ") or line.startswith("drop table "):
                  line=line.replace("drop view if exists ","")
                  line=line.replace("drop table if exists ","")
                  line=line.replace("drop view ","")
                  line=line.replace("drop table ","")
                  line=line.replace(";","").strip()
                  if line not in tbl:
                    if self.convert_to_env=='dev':
                      tbl.append(line)
                    else:
                      if line.startswith("gpo.mbse_") or line.startswith("dev.mbse_"):
                        tbl.append(line)
    txt=''
    for i in range(len(tbl)):
      name=(tbl[i].split('.')[1])
      txt=txt+ch+"'"+name+"'"+col_name+" from "+tbl[i]
      if i<len(tbl)-1:
        txt=txt+"\n"+"union \n"
      else:
        txt=txt+";"
    return txt


## Auxillary Functions to support XML to CSV class scripts

In [ ]:
def get_exact_field(x):
  res='' if x is None else x
  res=res.replace('%3A',':')
  res=res.replace('%2F','/')
  res=res.replace('%20',' ')
  result=res[:900]
  return result

def get_gc_platform(sym):
  return {
    'GC Golden Sample PD Cooking': 'PDx',
    'GC PD Cooking': 'Cooking',
    'GC PD Cross Category': 'Cross Category',
    'GC PD Dishwasher': 'Dishwasher',
    'GC PD KASA': 'KASA',
    'GC PD Laundry': 'Laundry',
    'GC PD Refrigeration': 'Refrigeration'}.get(sym, ' ')

def extract_from_primary_text(x):
  res =  re.sub(re.compile(r'<.*?>'), '', x)
  res = res.replace('&lt;', '<')
  res = res.replace('&gt;', '>')
  res = res.replace('&amp;','&')
  #res= re.sub(r'&[a-zA-Z]+?;', '', x)
  return res

def prefix_rqm_verification(x):
  return 'Verification_'+str(x)

def prefix_rqm_validation(x):
  return 'Verification_'+str(x)


def get_interface_platform(sym):
  return {
      'Cooking Product Internal Interfaces':'Cooking',
      'Cooking Product Internal System Interfaces':'Cooking',
      'Cooking Product External System Interfaces':'Cooking',
      'Cooking Product User Interfaces': 'Cooking',
      'Dishwasher Product Internal System Interfaces':'Dishwasher',
      'Dishwasher Product External System Interfaces':'Dishwasher',
      'Dishwasher Product User Interfaces': 'Dishwasher',
      'Interface - [Dryer] Product Internal Interfaces':'Dryers',
      'Clothes Washer Product User Interfaces':'Laundry',
      'Interface - [Laundry] Product External System Interfaces':'Laundry',
      'Interface - [HA Washer] Product Internal System Interfaces':'Laundry - HA',
      '[VA Washer] Interface - Product Internal System Interfaces' :'Laundry - VA',
      'Refrigerator Product Internal System Interfaces':'Refrigeration',
      'Refrigerator Product User Interfaces':'Refrigeration',
      'Refrigerator Product External System Interfaces':'Refrigeration',
      'Appliance External System Interfaces':'Cross'}.get(sym, ' ')

def extract_gcid(xml_path, schema_path):
  xml=xml_path.split('/')[-1]
  schema=schema_path.split('/')[-1]
  return ''.join(set(xml.replace('.xml',''))-set(schema.replace('.pkl','')))

def get_interface_type(sym):
  return {
      'Cooking Product External System Interfaces':'External',
      'Dishwasher Product External System Interfaces':'External',
      'Interface - [Laundry] Product External System Interfaces':'External',
      '[VA Washer] Interface - Product Internal System Interfaces':'Internal',
      'Cooking Product Internal Interfaces':'Internal',
      'Cooking Product Internal System Interfaces':'Internal',
      'Dishwasher Product Internal System Interfaces':'Internal',
      'Interface - [Dryer] Product Internal Interfaces':'Internal',
      'Interface - [HA Washer] Product Internal System Interfaces':'Internal',
      'Refrigerator Product Internal System Interfaces':'Internal',
      'Cooking Product User Interfaces':'User',
      'Dishwasher Product User Interfaces':'User',
      'Refrigerator Product User Interfaces':'User',
      'Clothes Washer Product User Interfaces':'User',
      'Refrigerator Product External System Interfaces':'External'}.get(sym, ' ')


def get_gcid_split(x):
  return '' if x is None else x.split('%2F')[-1]


def get_created_on(x):
    #return datetime.today().strftime("%Y-%m-%d %H:%M:%S")
    return (datetime.today() + timedelta(hours=5,minutes=30)).strftime("%Y-%m-%d")


def get_accesskey_gc_config(x):
    return 'EESCIBDW'


def get_accessvalue_gc_config(x):
    return 'Y2c0YlIzZHY'


def get_voe_req_type(sym):
  sym=sym.split('}')[-1]
  for key in {
      'dng_merged_VoERequirement1_identifier':'VoE',
      'dng_merged_PDComponentRequirement1_identifier':'PD Component',
      'dng_merged_PDModuleRequirement1_identifier':'PD Module',
      'dng_merged_PDSubsystemRequirement1_identifier':'PD Subsystem'
  }.keys():
      if sym in key:
          return {
              'dng_merged_VoERequirement1_identifier':'VoE',
              'dng_merged_PDComponentRequirement1_identifier':'PD Component',
              'dng_merged_PDModuleRequirement1_identifier':'PD Module',
              'dng_merged_PDSubsystemRequirement1_identifier':'PD Subsystem'
          }[key]
  return ''



def get_vox_req_type(sym):
  sym=sym.split('}')[-1]
  for key in {
      'dng_merged_ConsumerStakeholderNeed1_identifier':'Consumer Stakeholder Need',
      'dng_merged_GeneralStakeholderNeed1_identifier':'General Stakeholder Need',
      'dng_merged_StakeholderRequirement1_identifier':'Stakeholder Requirement'
  }.keys():
      if sym in key:
          return {
              'dng_merged_ConsumerStakeholderNeed1_identifier':'Consumer Stakeholder Need',
              'dng_merged_GeneralStakeholderNeed1_identifier':'General Stakeholder Need',
              'dng_merged_StakeholderRequirement1_identifier':'Stakeholder Requirement'
          }[key]
  return ''


def get_blank_values(x):
  return ''


## Class to extract any xml to csv with or without schema

In [ ]:
class rational_to_csv:
  def __init__(self,url,filename,schema_dictionary='',process_gcid='',populate_column=False,column_to_populate=''):
    self.d_ict=schema_dictionary
    self.gcid=process_gcid
    if process_gcid=='':
      self.url=url
    else:
      self.url=url.format(process_gcid)
    self.pattern=populate_column
    self.filename=filename
    self.colname=column_to_populate

  def get_headers(self,xml_schema):
    headers=[]
    for val in xml_schema.values():
      if isinstance(val,list):
        for ele in val:
          if isinstance(ele,tuple):
            if "_get_tag" in ele[0]:
              headers.append(ele[0].replace('_get_tag',''))
            else:
              headers.append(ele[0])
          else:
            if "_get_tag" in ele:
              headers.append(ele.replace('_get_tag',''))
            else:
              headers.append(ele)
      elif isinstance(val,tuple):
        if "_get_tag" in val[0]:
          headers.append(val[0].replace('_get_tag',''))
        else:
          headers.append(val[0])
      else:
        headers.append(val)
    return headers


  def append_subset_xml(self,node,tagvalue):
    if isinstance(tagvalue, tuple):
      new_tag_name, mapper = tagvalue
      if "get_tag" in new_tag_name:
        new_tag_name=new_tag_name.replace('_get_tag','')
        new_child = et.Element(new_tag_name)
        new_child.text = str(mapper(node.tag)) if callable(mapper) else node.tag
      else:
        new_child = et.Element(new_tag_name)
        new_child.text = str(mapper(node.text)) if callable(mapper) else node.text
    else:
      new_child = et.Element(tagvalue)
      new_child.text = node.text
    return new_child


  def create_pkl(self):
    self.xml_schema=pickle.dump(self.d_ict)
    return self.xml_schema



  def write_file(self,node):
    headers = self.get_headers(self.d_ict)
    processed_rows = set()
    rows_to_write = []
    for child in node:
      row = [child.find(tag_name).text if child.find(tag_name) is not None else '' for tag_name in headers]
      if tuple(row) not in processed_rows:
        rows_to_write.append(row)
        processed_rows.add(tuple(row))
    df = pd.DataFrame(rows_to_write, columns=headers)

    if not df.empty:
      if not self.pattern==True:
        df[self.colname] = self.gcid
    df.to_csv(self.filename+".csv",index=False)


  def xml_processing(self):
    if self.d_ict=='':
      xml_data=(requests.get(self.url, auth = ('EESCIBDW','cg4bR3dv'))).text
      df = pd.read_xml(xml_data)
      df.to_csv(self.filename+".csv",index=False)
    else:
      xml_data=(requests.get(self.url, auth = ('EESCIBDW','cg4bR3dv'))).content
      #xml_schema=self.create_pkl()
      xml_schema=self.d_ict
      root = et.fromstring(xml_data)
      subset_root = et.Element('root')
      for child in root:
        new_result = et.Element('root')
        for sub_child in child:
          tag = sub_child.tag.split('}')[1]
          for item in xml_schema:
            if isinstance(item,tuple):
              for t in item:
                if tag.startswith(t) and not tag.startswith(t+"_") and sub_child.text is not None:
                  tag_value = next(xml_schema[key] for key in xml_schema.keys() if t in key)
                  if isinstance(tag_value,list):
                    for ele in tag_value:
                      new_result.append(self.append_subset_xml(sub_child,ele))
                      if len(new_result) > 0:
                        subset_root.append(new_result)
                  else:
                    new_result.append(self.append_subset_xml(sub_child,tag_value))
                    if len(new_result) > 0:
                      subset_root.append(new_result)
            else:
              if tag.startswith(item) and not tag.startswith(item+"_"):
                tag_value = xml_schema[item]
                if isinstance(tag_value,list):
                  for ele in tag_value:
                    new_result.append(self.append_subset_xml(sub_child,ele))
                    if len(new_result) > 0:
                      subset_root.append(new_result)
                else:
                  new_result.append(self.append_subset_xml(sub_child,tag_value))
                  if len(new_result) > 0:
                    subset_root.append(new_result)


      if not subset_root:
          print('no data in xml')
          return

      self.write_file(subset_root)

# Class & Script Usage

In [ ]:
sriyash=code_conversions(convert_to_env='prd',replace_converted_script=True,populate_dateparm=False,expand_bulk_load_func=False,
                        expand_table_setup_func=False,
                         unload_src='prd',unload_s3='prd',unload_add_created_on=True,
                         unload_d_ate='',unload_dynamic_timestamp=False)

#_,ddl,_=sriyash.get_load_ddl_unload()
#sriyash.write_file(load,'load_gpo_new')
#tst=sriyash.find_max_created_on()
#sriyash.write_file(ddl,'additional_ddl_REQ1987761')
#sriyash.remove_dup_files()
sriyash.create_exec_file()
#sriyash.combine_files("simulation")
#sriyash.extract_error_report()


[]
[]


In [ ]:
url_voe_new='https://jrs.whirlpool.oncloudone.com/rs/query/25976/dataservice?report=25957&limit=-1&oslc_config.context=https%3A%2F%2Frelm.whirlpool.oncloudone.com%2Fgc%2Fconfiguration%2F1529&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=bcbb52b6b49679c0512891969de42147&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=c3a82e3ea637c92ea3851464ac3538c3&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=cb56864e379797820f57b21e32257928&pName=50f6e2629af1dfee87a70bb43d7b4953&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_PO7oQPQDEemgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_TjmcUOwJEemgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_ZEUBAHP_EeqKquSH-KPQfw&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_YgU4UJKhEemiHrXVdsPwwg&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_CFjdcIrZEeujvIKcmbFanA&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_Xm1ZkLMkEemSHLJbm-CIRQ&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_VvN_4M8sEemNlL0kGqqbtQ&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_BK2Z0JaKEempLNJRu3K9Ug&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_fgWXYAUREeqgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F__EPRYAUQEeqgmMjgllbi-Q&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_zAXIcAUTEeqhSMKisZXOXg&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_-J4SAPOMEei6VdHimkJ1ZA&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_MIQeINEWEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_Q-7wcP_vEemhSMKisZXOXg&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_xrnEENTiEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_4psBwNToEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_jgTF0dTrEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_Y9enUNTrEeefatrBpRDqyw&pVal=http%3A%2F%2Fjazz.net%2Fns%2Flqe%2Fmerge%2Fgensym%2Frm%2FVoE%2520Requirement%2520Set&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.approved&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.new&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23DefaultWorkflow.state.s1&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.complete&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.underreview&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FConceptValidation&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FDesignValidation&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FProductValidation&pVal=http%3A%2F%2Fjazz.net%2Fxmlns%2Fprod%2Fjazz%2Frqm%2Fprocess%2F1.0%2FWorkflowState%2Fcom.ibm.rqm.planning.common.new&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FDraft&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FSubmitted&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FInProgress&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FTestCompleted&pVal=*&pVal=*&basicAuthenticationEnabled=true'

url_voe_old='https://jrs.whirlpool.oncloudone.com/rs/query/24671/dataservice?report=24652&limit=-1&oslc_config.context=https%3A%2F%2Frelm.whirlpool.oncloudone.com%2Fgc%2Fconfiguration%2F1529&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=ProjectIds&pName=bcbb52b6b49679c0512891969de42147&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=a543f326cc98f206e7ff15d75bb0544f&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=1cf42e38d53ea40eb4d8ce5ad50a5cb3&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=76cbbb0d4698babfe646ae0691fb71e0&pName=cb56864e379797820f57b21e32257928&pName=50f6e2629af1dfee87a70bb43d7b4953&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_PO7oQPQDEemgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_TjmcUOwJEemgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_ZEUBAHP_EeqKquSH-KPQfw&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_YgU4UJKhEemiHrXVdsPwwg&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_CFjdcIrZEeujvIKcmbFanA&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_Xm1ZkLMkEemSHLJbm-CIRQ&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_VvN_4M8sEemNlL0kGqqbtQ&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_BK2Z0JaKEempLNJRu3K9Ug&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F_fgWXYAUREeqgmMjgllbi-Q&pVal=https%3A%2F%2Frm2.whirlpool.oncloudone.com%2Frm2%2Fprocess%2Fproject-areas%2F__EPRYAUQEeqgmMjgllbi-Q&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_zAXIcAUTEeqhSMKisZXOXg&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_-J4SAPOMEei6VdHimkJ1ZA&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_MIQeINEWEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_Q-7wcP_vEemhSMKisZXOXg&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_xrnEENTiEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_4psBwNToEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_jgTF0dTrEeefatrBpRDqyw&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2Fqm%2Fprocess%2Fproject-areas%2F_Y9enUNTrEeefatrBpRDqyw&pVal=http%3A%2F%2Fjazz.net%2Fns%2Flqe%2Fmerge%2Fgensym%2Frm%2FVoE%2520Requirement%2520Set&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.approved&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.new&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23DefaultWorkflow.state.s1&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.complete&pVal=http%3A%2F%2Fwww.ibm.com%2Fxmlns%2Frdm%2Fworkflow%2FDefaultWorkflow%23com.ibm.rdm.workflow.common.underreview&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FConceptValidation&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FDesignValidation&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FProductValidation&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FDraft&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FSubmitted&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FInProgress&pVal=https%3A%2F%2Fqm.whirlpool.oncloudone.com%2FCategoryValue%2FTestCompleted&pVal=*&pVal=*&basicAuthenticationEnabled=true'

#url=[url0,url1,url2]
name=['url0','url1','url2']


#for i in range(len(url)):
sriyash=rational_to_csv(url=url_voe_new,filename='url_voe_new',
                      schema_dictionary='',process_gcid='',
                      populate_column=False,column_to_populate='')
sriyash.xml_processing()

In [ ]:
import pandas as pd
import datetime as dt

df=pd.read_csv("Error.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'Error.csv'

In [ ]:
def get_attributes_connexus(strng):
  cleaned=strng.replace("[",'').replace("]",'').replace('{','').replace('}','').replace('"S":','')
  cleaned=cleaned.replace('"M":','').replace('"','').split(',')
  #Get all the attribute list
  attr_list=[]
  for ele in cleaned:
    if ele.split(":")[0] not in attr_list:
      attr_list.append(ele.split(":")[0])

  #Get all parameters in attrName
  param_list=[]
  max_list=[]
  min_list=[]
  value_list=[]
  for ele in cleaned:
    if ele.split(":")[0] == 'attrName' and ele.split(":")[1] not in param_list:
      param_list.append(ele.split(":")[1])

  #Get all max_alert in attrName
    if ele.split(":")[0] == 'attrMaxAlertRange' and ele.split(":")[1] not in param_list:
      max_list.append(float(ele.split(":")[1]))

  #Get all min_alert in attrName
    if ele.split(":")[0] == 'attrMinAlertRange' and ele.split(":")[1] not in param_list:
      min_list.append(float(ele.split(":")[1]))

  #Get all value in attrName
    if ele.split(":")[0] == 'attrValue' and ele.split(":")[1] not in param_list:
      value_list.append(float(ele.split(":")[1]))

  print(param_list)
  col_names=[]
  final={}
  print(value_list)
  for i in range(len(param_list)):
    col_names.append(param_list[i])
    col_names.append(param_list[i]+"_exceeding_range")

    final[param_list[i]]=[value_list[i]]
    val=value_list[i]
    min_val=min_list[i]
    max_val=max_list[i]
    if val>=min_val and val<=max_val:
      final[param_list[i]+"_status"]=[0]
    else:
      final[param_list[i]+"_status"]=[1]

  final=pd.DataFrame(final)
  return final




def add_rows_connexus(df):
  fin_df=pd.DataFrame()
  for i in range(df.shape[0]):
    fin_df=pd.concat([fin_df,get_attributes_connexus(df.loc[i,"attributeInfo"])])
  df_new=pd.concat([df.reset_index(drop=True),fin_df.reset_index(drop=True)],axis=1)
  df_new.drop(columns=['PK','SK','attributeInfo','batchEndTime','batchId',
                   'batchStartTime','createdAt','machineName','orgName',
                   'plantName','processName','totalCycleDuration'],inplace=True,axis=1)
  b = [0,10,18,24]
  l = ['Morning','Evening','Night']
  df_new['timeLog']=pd.to_datetime(df_new['timeLog']).dt.hour
  df_new['timeLog'] = pd.cut(df_new['timeLog'], bins=b, labels=l, include_lowest=True)
  df_new.to_csv("Final.csv",index=False)
  return df_new


In [ ]:
def get_attributes_gallus(strng):
  try:
    cleaned=strng.replace("[",'').replace("]",'').replace('{','').replace('}','').replace('"S":','')
    cleaned=cleaned.replace('"M":','').replace('"','').split(',')
    #Get all the attribute list
    attr_list=[]
    for ele in cleaned:
      if ele.split(":")[0] not in attr_list:
        attr_list.append(ele.split(":")[0])

    #Get all parameters in attrName
    param_list=[]
    value_list=[]
    for ele in cleaned:
      if ele.split(":")[0] == 'attrName' and ele.split(":")[1] not in param_list:
        param_list.append(ele.split(":")[1])

    #Get all value in attrName
      if ele.split(":")[0] == 'attrValue' and ele.split(":")[1] not in param_list:
        try:
          value_list.append(float(ele.split(":")[1]))
        except:
          value_list.append(ele.split(":")[1])

    col_names=[]
    final={}
    for i in range(len(param_list)):
      col_names.append(param_list[i])
      final[param_list[i]]=[value_list[i]]
      final=pd.DataFrame(final)
    return cleaned
  except:
    return pd.DataFrame()


def add_rows_gallus(df):
  fin_df=pd.DataFrame()
  for i in range(df.shape[0]):
    fin_df=pd.concat([fin_df,get_attributes_gallus(df.loc[i,"attributeInfo"])])
  df_new=pd.concat([df.reset_index(drop=True),fin_df.reset_index(drop=True)],axis=1)
  df_new.drop(columns=['PK','SK','attributeInfo','batchEndTime','batchId',
                   'batchStartTime','createdAt','emergencySW','equipmentNumber','loggedinUser','orgName',
                   'processName','refreshRate','totalCycleDuration'],inplace=True,axis=1)
  # b = [0,10,18,24]
  # l = ['Morning','Evening','Night']
  # df_new['timeLog']=pd.to_datetime(df_new['timeLog']).dt.hour
  # df_new['timeLog'] = pd.cut(df_new['timeLog'], bins=b, labels=l, include_lowest=True)
  df_new.to_csv("Final.csv",index=False)
  return df_new

In [ ]:
l_ist=get_attributes_gallus(df.loc[0,'allErrors'])

NameError: name 'df' is not defined

In [ ]:
l_ist[0].split(':',1)[0]

NameError: name 'l_ist' is not defined

In [ ]:
':' in l_ist[911]

NameError: name 'l_ist' is not defined

In [ ]:
df={
  'timeOn':[],
  'errorText':[],
  'errorNo':[],
  'moduleType':[],
  'module':[],
  'constructionUnit':[],
  'category':[],
  'timeOff':[]
}

for i in range(len(l_ist)):
  if ':' in l_ist[i]:
    df[l_ist[i].split(':',1)[0]].append(l_ist[i].split(':',1)[1])
  else:
    df['timeOff'].append('2023-09-11T00:25:15.576+02:00')

len(df['timeOn'])
len(df['errorText'])
len(df['errorNo'])
len(df['moduleType'])
len(df['module'])
len(df['constructionUnit'])
len(df['category'])
len(df['timeOff'])
pd.DataFrame(df).to_csv("Error_Report.csv")


NameError: name 'l_ist' is not defined

In [ ]:
add_rows_gallus(df)

AttributeError: 'dict' object has no attribute 'shape'

In [ ]:
import pandas as pd
ana_df=pd.read_csv("Final.csv")
ana_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Final.csv'

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.tree import DecisionTreeClassifier
from matplotlib import pyplot as plt
from sklearn import tree

df_dumm = ana_df.drop(columns=["timeLog",
                "Reactor Temperature_status","Reactor Pressure_status",
                "Seperator Temperature_status","Seperator Pressure_status",
                "Heating Temperature_status","Oee_status","Mtbf_status",
                "Mttr_status","Machine in breakdown_status"],axis=1)
Y=df_dumm['emergencySW']
X=df_dumm.drop(columns="emergencySW",axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)

In [ ]:
clf = DecisionTreeClassifier(random_state=0).fit(X_train, y_train)

In [ ]:
classification_report(y_test, clf.predict(X_test))

In [ ]:
fig = plt.figure(figsize=(25,20))
_ = tree.plot_tree(clf,
                   feature_names=X.columns,
                   class_names=["0","1"],
                   filled=True)

In [ ]:
fig.savefig("decistion_tree.png")

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Read the CSV file
df = pd.read_csv('Final_1.csv')

# Convert the 'Date' column to a datetime object if it's not already
df['timeLog'] = pd.to_datetime(df['timeLog'])

# Define the date range from 26th Aug 2023 to 25th Sep 2023
import pandas as pd
from datetime import datetime, timedelta

# Define the date range from 26th Aug 2023 to 25th Sep 2023
start_date = datetime(2023, 8, 26)
end_date = datetime(2023, 9, 25)

# Initialize counters and other parameters
job_counter = 2000
partial_job_counter = random.randint(500, 2500)
production_counter = 15000
total_counter = 557783

# Create lists to store the extrapolated data
extrapolated_data = {
    'machineName':[],
    'machineStatus':[],
    'machineType':[],
    'plantName':[],
    'smartbearStatus':[],
    'Date': [],
    'Job Counter': [],
    'Partial Job Counter': [],
    'Production Counter': [],
    'Total Counter': [],
}

# Iterate through the date range
for date in pd.date_range(start=start_date, end=end_date, freq='D'):
    for minute in range(1440):  # 24 hours * 60 minutes
        time = date + timedelta(minutes=minute)

        partial_job_counter = random.randint(500, 2500)

        # Check if job_counter needs to reset to 2000
        if partial_job_counter > job_counter:
            job_counter += random.randint(25, 125)

        # Check if production_counter needs to reset to 15000
        if production_counter > 30000:
            production_counter = 15000

        # Update production_counter based on job_counter
        production_increment = random.randint(125, 375)
        production_counter += production_increment

        # Update total_counter based on production_counter
        total_increment = random.randint(25, 75)
        total_counter += total_increment

        # Append the data to the extrapolated_data dictionary
        extrapolated_data['machineName'].append('GL-LM440-0156')
        extrapolated_data['machineStatus'].append(random.choices([2,3,5],weights=[0.48,0.35,0.17])[0])
        extrapolated_data['machineType'].append('LM 440')
        extrapolated_data['plantName'].append('Pune')
        extrapolated_data['smartbearStatus'].append(2)
        extrapolated_data['Date'].append(time.strftime('%Y-%m-%d %H:%M'))
        extrapolated_data['Job Counter'].append(job_counter)
        extrapolated_data['Partial Job Counter'].append(partial_job_counter)
        extrapolated_data['Production Counter'].append(production_counter)
        extrapolated_data['Total Counter'].append(total_counter)

# Create a DataFrame from the extrapolated data
extrapolated_df = pd.DataFrame(extrapolated_data)

# Save the extrapolated data to a new CSV file
extrapolated_df.to_csv('extrapolated_data.csv', index=False)